# Speed violations in Chicago Part II

In this lab session, we quickly wrangle a dataset and then move to Tableau.

The process:
  * We add the data on the actual crashes/injuries (which was scrapped from a PDF).
  * We take the data on the amount of speed violations from last class.

We import the pandas library for handling dataframes, Geocoder to map addresses, time to avoid query_overloads. 
We also enable multiple outputs.

In [1]:
import pandas as pd
from pygeocoder import Geocoder
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load and clean the dataset that contains actual incidents in Chicago

We load the dataset with the incidents. The dataset was scrapped from a PDF and contains spaces that we do not want.
We set the separator to remove the whitespaces.

In [2]:
incidents = pd.read_csv('incidents.csv', sep='\s*;\s*', header=0, engine='python')

The file also contains a lot of empty rows, which we remove. For merging this data with other datasets, we capitalize the `Address` column.

In [3]:
incidents = incidents.dropna(how='any')
incidents['Address'] = incidents['Address'].str.upper() 

We delete the `Unnamed: 0` column and polish the Name column

In [4]:
del incidents['Unnamed: 0']
incidents.rename(columns = {'Name 1':'Name'}, inplace = True)

## Obtain the coordinates for the incidents

The file with the incidents contains addresses but not coordinates. We need the coordinates to map the incidents.

An example of obtaining the coordinates from `Geocoder`.

In [ ]:
results = Geocoder.geocode('2620 W TOUHY AVE')
results.coordinates

We generate a list of all the addresses and add Chicago and Illinois to ensure that we get the correct coordinates.

In [17]:
addresses = (incidents['Address'] + ',' + 'Chicago' + ',' + 'IL').tolist()

Google has query limits, so be careful that you do not violate them. Here, we play it easy and get the coordinates for on address at a time and store it. We backup the dataframe regularly. ** DO NOT RUN THIS PART IN CLASS - THE RESULT IS IN THE LAB FOLDER! **

In [ ]:
codes = pd.DataFrame()
geocode = dict()
for address in addresses:
    geocode['Address'] = address
    geocode['Location'] = Geocoder.geocode(address).coordinates
    codes = codes.append(geocode, ignore_index=True)
    if len(codes) % 100 == 0:
        codes.to_csv('codes.csv')
    time.sleep(1)
codes.to_csv('codes_clean.csv')

We load the coordinates from the file and clean the file.

In [5]:
codes_clean = pd.read_csv('codes_clean.csv')
#del codes_clean['Unnamed: 0']
#codes_clean['Address'] = codes_clean.Address.str.replace(',Chicago,IL', '')
codes_clean['Latitude'], codes_clean['Longitude'] = codes_clean['Location'].str.split(',',1).str
codes_clean['Latitude'] = codes_clean['Latitude'].str.replace('(','')
codes_clean['Longitude'] = codes_clean['Longitude'].str.replace(')','')

We merge the coordinates with the incidents.

In [6]:
incidents = incidents.merge(codes_clean, how='left', left_on='Address', right_on='Address')

We save the file.

In [7]:
incidents.to_csv('incidents_clean.csv', index=False)

We load the data from last lab (**ADAPT THE PATH TO YOUR OS**). We use the location as unique identifier.

In [8]:
violation = pd.read_csv('../Lab Session 4/tableau.csv')
violation['Location'] = violation.apply(lambda x:'(%s, %s)' % (x['LATITUDE'],x['LONGITUDE']),axis=1)

We save the violations.

In [9]:
violation.to_csv('violations.csv', index=False)

We develop a dataframe that only contains the locations from the incidents and the violations. This will help us to map it in Tableau later.

In [10]:
locations_violations = violation[['LATITUDE','LONGITUDE', 'Location']]
locations_violations = locations_violations.rename(columns={'LATITUDE':'Latitude','LONGITUDE':'Longitude'})
locations_incidents = incidents[['Location', 'Longitude','Latitude']]

The violations contain multiple records for the same location. We drop them.

In [11]:
locations =pd.concat([locations_violations, locations_incidents], ignore_index=True)
locations = locations.drop_duplicates()
len(locations)

1615

We obtain the zip codes for the locations. This enables us to use zip codes to analyze the situation.

In [17]:
zip_codes = pd.DataFrame()
zip_code = dict()
for index,location in locations.iterrows():
    zip_code['Location'] = location['Location']
    zip_code['ZIP'] = Geocoder.reverse_geocode(float(location['Latitude']),float(location['Longitude'])).postal_code
    zip_codes = zip_codes.append(zip_code, ignore_index=True)
    if len(zip_codes) % 10 == 0:
        zip_codes.to_csv('zip_codes.csv')
    time.sleep(1)
zip_codes.to_csv('zip_codes.csv')

We merge the zip codes with the locations.

In [20]:
locations = locations.merge(zip_codes)
locations

,Latitude,Location,Longitude,ZIP
0,41.8666,"(41.866585283, -87.698962375)",-87.699,60612
1,41.7564,"(41.756374355, -87.633816585)",-87.6338,60620
2,41.6907,"(41.690701951, -87.664122385)",-87.6641,60643
3,41.9533,"(41.953329545, -87.764267264)",-87.7643,60641
4,41.9241,"(41.924128491, -87.762993999)",-87.763,60639
5,41.9382,"(41.938181259000004, -87.787655485)",-87.7877,60634
6,41.8809,"(41.880938186, -87.71789844899997)",-87.7179,60624
7,41.9557,"(41.955725645, -87.76693812299999)",-87.7669,60634
8,41.691,"(41.691025456, -87.66424769)",-87.6642,60643
9,41.9623,"(41.96226556, -87.684566996)",-87.6846,60625


An example for reverse geocoding.

In [12]:
results = Geocoder.reverse_geocode(locations['Latitude'][0],locations['Longitude'][0]).postal_code
results

'60612'

In [22]:
locations.to_csv('locations.csv', index=False)